In [ ]:
import os
import urllib.request
import librosa
import soundfile
import numpy as np
from flask import Flask, flash, request, redirect, render_template
from werkzeug.utils import secure_filename
import pickle
import pandas as pd
import pyaudio
import wave
import time

with open(f'model/modelv9.pkl', 'rb') as f:
    model = pickle.load(f)

UPLOAD_FOLDER = './uploads'

app = Flask(__name__)
app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024

ALLOWED_EXTENSIONS = set(['wav'])


def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft=np.abs(librosa.stft(X))
    result=np.array([])
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0)
    result=np.hstack((result, mfccs))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
    return result

def allowed_file(filename):
	return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
	
@app.route('/')
def upload_form():
	return render_template('upload.html', intro=True)

@app.route('/', methods=['POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('Invalid File')
            return render_template('upload.html', intro=False)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            flash('File successfully uploaded')
            feat = extract_feature(myfile)
            feat = feat.reshape(1,-1)
            prediction = model.predict(feat)
            prediction = prediction[0].split('_')
            
            return  render_template('upload.html', result=True, gender=prediction[1], mood=prediction[0])
        else:
            flash('Allowed file type is just wav for now')
            return render_template('upload.html', intro=False)

@app.route('/a', methods=['POST'])
def record():
    moment=time.strftime("%Y-%b-%d__%H_%M_%S",time.localtime())
    chunk = 1024  # Record in chunks of 1024 samples
    sample_format = pyaudio.paInt16  # 16 bits per sample
    channels = 2
    fs = 44100  # Record at 44100 samples per second
    seconds = 5
    filename = f'{moment}.wav'

    p = pyaudio.PyAudio()  # Create an interface to PortAudio

    print('Recording')

    stream = p.open(format=sample_format,
                    channels=channels,
                    rate=fs,
                    frames_per_buffer=chunk, input=True)

    frames = []  # Initialize array to store frames

    # Store data in chunks for 3 seconds
    for i in range(0, int(fs / chunk * seconds)):
        data = stream.read(chunk)
        frames.append(data)

    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()

    print('Finished recording')

    # Save the recorded data as a WAV file
    wf = wave.open(os.path.join(app.config['UPLOAD_FOLDER'], filename), 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(p.get_sample_size(sample_format))
    wf.setframerate(fs)
    wf.writeframes(b''.join(frames))
    wf.close()
    myfile = os.path.join(app.config['UPLOAD_FOLDER'], filename)
    feat = extract_feature(myfile)
    feat = feat.reshape(1,-1)
    prediction = model.predict(feat)
    prediction = prediction[0].split('_')
    return  render_template('upload.html', result=True, gender=prediction[1], mood=prediction[0])

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Apr/2020 02:03:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:03:17] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:03:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2020 02:04:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:04:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:04:23] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2020 02:04:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:05:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:06:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:06:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2020 02:06:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:06:36] "GET /favicon.ico HTTP/1.1" 404 -


Recording
Finished recording


127.0.0.1 - - [11/Apr/2020 02:06:42] "POST /a HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:06:43] "GET / HTTP/1.1" 200 -


Recording
Finished recording


...

127.0.0.1 - - [11/Apr/2020 02:25:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:26:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:27:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:32:46] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2020 02:32:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:32:52] "HEAD / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:32:52] "HEAD / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:32:52] "HEAD / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:33:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:33:34] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [11/Apr/2020 02:34:22] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:38:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [11/Apr/2020 02:46:15] "GET / HTTP/1.1" 200 -


Recording
Finished recording


127.0.0.1 - - [11/Apr/2020 02:46:23] "POST /a HTTP/1.1" 200 -


Recording
Finished recording


127.0.0.1 - - [11/Apr/2020 02:46:44] "POST /a HTTP/1.1" 200 -
